# Temperature Plots (all venues)


Choose your venue from the dropdown menu.  Then you can use the slider below the plot to explore your data.  You can make the data "window" as small or as large as you want, and also slide the entire window to the left or the right.  You can also choose a window size to be an hour, a day, a week, or a year using the buttons above the plot.

These plots work best on a large screen.  If you can't see the right edge of the plot, try using your browser zoom features to get the full plot.  On many browsers, ctrl-+ (holding the control button and the + sign at the same time) will zoom in and ctrl-minus (the control button with the minus sign) will zoom out.  Zooming is also available from the browser menus. We are looking at what we can do to facilitate use on other screen sizes.

```{admonition} Bug work-around
If you change venue sometimes you will get the new data superimposed on the old data rather than replacing it.  If this happens, reload the page. 
```


In [ ]:
# Imports 
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go   
from IPython.display import display

import datetime
# import pytz


# Get the possible data venues
venuekeysfile = "venue-keys.csv"
dfVenueKeys = pd.read_csv(venuekeysfile)
dfVenueKeys = dfVenueKeys.dropna(subset=['channel_id'])

#give user option to select their venue
venueDropdown = widgets.Dropdown(
    options=dfVenueKeys['venue_id'],
    value=dfVenueKeys['venue_id'][0],
    description='Venue ID:',
    disabled=False,
)


container = widgets.HBox(children=[venueDropdown])

print(venueDropdown.value)

#Retrieve the venue and begin graphing
dfCollatedDataSet = pd.DataFrame(columns=['timestamp', 'entry_id', 'temperature', 'rh', 'voltage', 'venue_id'])
for index, venueSensorDetails in dfVenueKeys.iterrows():
    
    sensorMacOfSelection = venueSensorDetails['sensor_MAC']
    venueOfSelection = str(venueSensorDetails['venue_id'])
    dfTempDataSet = pd.read_csv('deviceData/'+ "venue_" + venueOfSelection + "_with_device_" + sensorMacOfSelection + '.csv' )
    dfTempDataSet['timestamp'] = pd.to_datetime(dfTempDataSet['timestamp'])
    dfTempDataSet['timestamp'] = dfTempDataSet['timestamp'].dt.tz_convert("Europe/London")
    dfTempDataSet['venue_id'] = venueSensorDetails['venue_id']
    dfTempDataSet = dfTempDataSet[(dfTempDataSet.temperature <= 100)]
    dfTempDataSet = dfTempDataSet[(dfTempDataSet.temperature > -10)] # eliminate rogue data
    print(venueSensorDetails['venue_id'])
    # %z give +0000 or +0100, and .str[2] gets the third char of that string, the 0 or 1
    # 6000 is ms in an hour.
    print(dfTempDataSet['timestamp'][1])
    # demonstrates that it does get daylight savings right - with +00:00 or +01:00
    # however, plotly can't plot them both on the same trace. See https://github.com/plotly/plotly.py/issues/2872
    # The approach is to create two traces where the dates are faked - anything in summer
    # is given as an hour ahead of reality on the first trace and that's used to plot the points,
    # but the second trace is used to put up the hover text which clarifies what the timezone
    # offset on that point is.   Then the
    # x axis has to be rendered without showing the time zone.

    # %z gets +0x00 where x is the number of hours ahead of "normal" the timezone is in.
    # add that number of hours to the time and then fail to show the offset.
    # :TODO: also do on RH; try to get rid of the concat warning.
    dfTempDataSet['offset'] = pd.to_numeric(dfTempDataSet['timestamp'].dt.strftime("%z").str[2])
    print(dfTempDataSet['offset'][1])
    dfTempDataSet['offset'] = dfTempDataSet['offset']
    dfTempDataSet['timedelta'] = dfTempDataSet['offset'].map(lambda x: pd.Timedelta(x,"h"))
    dfTempDataSet['timestamp'] = dfTempDataSet['timestamp'] + dfTempDataSet['timedelta']
    print(dfTempDataSet['timestamp'][1])
    # This is successfully the wrong time so we can plot it on a mixed summer time plot -
    # as long as we don't ever show the offset.

    # 
    dfCollatedDataSet = dfCollatedDataSet.append(dfTempDataSet, ignore_index=True)
    dfCollatedDataSet['timestamp'] = pd.to_datetime(dfCollatedDataSet['timestamp'])
    dfCollatedDataSet['timestamp'] =  dfCollatedDataSet['timestamp']

    
    print('Loading data for venue: ', venueSensorDetails['venue_id']) 
    


print('Check')
dfCollatedDataSet.sample(6)




# Assign an empty figure widget with two traces
trace0 = go.Scatter(customdata=dfCollatedDataSet[dfCollatedDataSet['venue_id'] == 0], 
                    y=dfCollatedDataSet['temperature'], 
                    x = dfCollatedDataSet['timestamp'], 
                    mode='lines', 
                    hoverinfo='all', 
                    name='Temperature',
                    )

# trace1 = go.Scatter(customdata=dfCollatedDataSet[dfCollatedDataSet['venue_id'] == 0], 
#                     y=dfCollatedDataSet['rh'], 
#                     x = dfCollatedDataSet['timestamp'], 
#                     mode='lines', 
#                     hoverinfo='all', 
#                     name='Relative Humidity',

# )


g = go.FigureWidget(data=trace0,
                    #data=[trace0, trace1],
                    layout = go.Layout(
                        yaxis=dict(range=[0,0])
                    ))


print("Job Done")



In [ ]:
updatemenu = []
buttons = []


# button with one option for each dataframe
for index, venue in dfVenueKeys.iterrows():
    buttons.append(dict(method='update',
                        label='Venue ' + str(venue['venue_id']),
                        visible=True,
                        args=[{'y':[dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['temperature'].values], #, 
                                    #dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['rh'].values],
                               'x':[dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['timestamp'].values],
                               'type':'scatter',   
                            #  'name': 'Temperature',
                                                          
                               },
                               {
                                   'title.text': 'Temperature for Venue = ' + str(venue['venue_id']),
                                   'title.font.color': 'green',
                                    ## KDs original for plotting RH and temp on same plot
                                    #'yaxis.range': [-5,dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']][['temperature', 'rh']].max().max()+5], 
                                    ## adjusting for data range, temp only
                                    #'yaxis.range': [dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['temperature'].min()-5,dfCollatedDataSet[dfCollatedDataSet['venue_id']==venue['venue_id']]['temperature'].max()+5],
                                    # fixed, because we get rogue readings on cheap devices
                                   'yaxis.range': [0,30],  
                                   'yaxis.title.text': 'Temperature',
                                   'xaxis.title.text': 'Timestamp'
                               },
                               ],
                        )
                  )




# some adjustments to the updatemenus
updatemenu = []
your_menu = dict()
updatemenu.append(your_menu)

updatemenu[0]['buttons'] = buttons
updatemenu[0]['direction'] = 'down'
updatemenu[0]['showactive'] = True
# updatemenu[0]['active'] = 0

 

fig = go.Figure(g)
# add dropdown menus to the figure
fig.update_layout(showlegend=True, 
              updatemenus=updatemenu, 
              autosize = True, 
              title= "Please select a venue to see your data.",
              width=1000, 
              height=500,
)

fig.update_layout(
    hovermode='x unified',
    hoverlabel=dict(
        bgcolor="white",
        # font_size=16,
        font_family="Rockwell"
    )
)

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(
                     label="All",
                     step="all"
                     ),
                                dict(count=1,
                     label="Hour",
                     step="hour",
                     stepmode="todate"),
                dict(count=1,
                     label="Day",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="Week",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="Year",
                     step="year",
                     stepmode="backward")
            ])
        ),
        rangeslider=dict(
            visible=True,
        ),
        type="date"
    )
)

#fig.update_xaxes(
#    tickformat="%Y-%m-%d %H:%M %Z"  # date format
#)

#fig.update_yaxes(range=[50, 60])  


fig.add_hline(y=16, annotation_text='16C - usual minimum for children', annotation_font_color="blue", line_color='red', layer='above', line_dash='dash')
# fig.update_yaxes(range = [-5, dfCollatedDataSet['temperature'].max()+5])
fig.show()
